# Richmeister Integration Test Setup

This book allows the installation of test infrastructure, including tables in two regions and installation and configuration of the replication functions for multi-master replication of the tables.

## Asset Deployment

In [ ]:
# Deploy to the east region
import os
deploy_east = os.environ['DEPLOY_BUCKET_EAST']

In [ ]:
%%script env DEPLOY_BUCKET="$deploy_east" bash
cd ../s2fq/
make package
make deploy

In [ ]:
# Deploy to the west
deploy_west = os.environ['DEPLOY_BUCKET_WEST']

In [ ]:
%%script env DEPLOY_BUCKET="$deploy_west" bash
cd ../s2fq/
make package
make deploy

## Test Table Creation

In [ ]:
import boto3

ddb_east = boto3.client('dynamodb', region_name='us-east-1')
ddb_west = boto3.client('dynamodb', region_name='us-west-2')

table_name = 'TestTable'

In [ ]:
create_table_east_response = ddb_east.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    },
    StreamSpecification={
        'StreamEnabled': True,
        'StreamViewType': 'NEW_AND_OLD_IMAGES'
    }
)

print create_table_east_response

In [ ]:
table_east_stream = create_table_east_response['TableDescription']['LatestStreamArn']
print table_east_stream

In [ ]:
create_table_west_response = ddb_west.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    },
    StreamSpecification={
        'StreamEnabled': True,
        'StreamViewType': 'NEW_AND_OLD_IMAGES'
    }
)

print create_table_west_response

In [ ]:
table_west_stream = create_table_west_response['TableDescription']['LatestStreamArn']
print table_west_stream

## Clean Up

In [ ]:
# Delete test tables
for client in [ddb_east, ddb_west]:
    response = client.delete_table(
        TableName = table_name
    )
    
    print response